<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=b96fac1d963e8b760cd80670e0305962a8e4025b8f49b5af952467e636a88998
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-26 13:17:31
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.38 K
Current:  1.37 C
-------------------
Today PnL: -1.75 L (-1.27%)
Current PnL: -22.76 L (-15.31%)
CY Booked + Current PnL: -11.32 L (-7.61%)
-------------------
Total profit:  83.89 K
Total loss:  -23.59 L
-------------------
Total Booked + Current PnL: 15.66 L (12.87%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.36%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 88.62 L (64.77%)
Deployed:  1.22 C
Current:  1.37 C
CAGR/XIRR %: 7.11%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,92.53,35.0,M-SC,2.74,82988.0,-17789.0,17884.0,-3.02,-17.65,21.55,0.09,245.0,-0.99,0.61,9.03,OX40N,NTT,DURABLES
14,BLUESTARCO,2080.00,9.48,47.0,H-MC,3.82,188890.0,24220.0,19116.0,-1.99,14.71,10.12,26.31,89.0,1.27,1.38,23.86,X40N,NTT,AC
50,NATIONALUM,244.55,-46.15,48.0,H-MC,10.91,106853.0,6349.0,22760.0,-1.95,6.32,21.30,28.96,79.0,0.28,0.78,42.43,MH,ATH,METALS
42,ITC,452.00,-38.77,45.0,H-LC,2.15,197972.0,-2166.0,23064.0,1.19,-1.08,11.65,10.44,4.0,-0.09,1.45,4.42,X40,NTT,FMCG
57,RELIANCE,1533.00,-14.04,45.0,H-LC,4.07,215327.0,4961.0,23815.0,0.58,2.36,11.06,13.68,37.0,0.21,1.57,19.27,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TATAMOTORS,1065.0,-22.16,43.0,M-LC,2.35,245551.0,-30972.0,141044.0,1.83,-11.20,57.44,39.81,54.0,-0.22,1.80,17.68,XY24,NTT,AUTO
42,ITC,452.0,-38.77,45.0,H-LC,2.15,197972.0,-2166.0,23064.0,1.19,-1.08,11.65,10.44,4.0,-0.09,1.45,4.42,X40,NTT,FMCG
45,KANSAINER,340.0,-67.61,54.0,H-SC,1.45,223263.0,-46404.0,82741.0,1.15,-17.21,37.06,13.47,138.0,-0.56,1.63,12.92,XY24,NTT,PAINTS
57,RELIANCE,1533.0,-14.04,45.0,H-LC,4.07,215327.0,4961.0,23815.0,0.58,2.36,11.06,13.68,37.0,0.21,1.57,19.27,XY25,NTT,REFINERIES
61,SAMMAANCAP,326.0,-171.59,56.0,M-SC,5.87,82068.0,-20142.0,113533.0,0.27,-19.71,138.34,91.37,208.0,-0.18,0.60,33.22,XY25,NTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
46,KPIGREEN,731.05,-4.41,30.0,H-SC,8.74,114055.0,-11242.0,70167.0,-3.81,-8.97,61.52,47.03,141.0,-0.16,0.83,42.81,MH,ATH,POWER
60,SAIL,228.00,44.35,54.0,M-MC,11.15,227425.0,2463.0,165202.0,-3.26,1.09,72.64,74.53,192.0,0.01,1.66,33.70,XY24,BTT,STEEL
32,HINDZINC,730.22,20.84,53.0,M-LC,2.78,196729.0,-8347.0,120910.0,-3.11,-4.07,61.46,54.89,52.0,-0.07,1.44,19.04,X5K,ATH,METALS
27,HAPPSTMNDS,1488.71,-20.51,27.0,H-SC,10.53,85776.0,-42389.0,152415.0,-3.06,-33.07,177.69,85.85,132.0,-0.28,0.63,0.00,AR,ATH,IT
77,TTKPRESTIG,770.00,92.53,35.0,M-SC,2.74,82988.0,-17789.0,17884.0,-3.02,-17.65,21.55,0.09,245.0,-0.99,0.61,9.03,OX40N,NTT,DURABLES


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BANKINDIA,190.00,-29.19,54.0,H-MC,8.21,181422.0,1614.0,110794.0,-1.50,0.90,61.07,62.52,88.0,0.01,1.33,35.29,XR,NTT,BANKS
52,PGHH,17973.08,-30.15,52.0,H-MC,3.40,202365.0,1545.0,67226.0,-2.14,0.77,33.22,34.25,80.0,0.02,1.48,6.01,X40,ATH,FMCG
86,WIPRO,420.00,-14.70,38.0,M-LC,5.50,150040.0,-905.0,110354.0,-1.06,-0.60,73.55,72.51,53.0,-0.01,1.10,5.39,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,90.20,47.0,H-SC,10.15,120447.0,-22032.0,41554.0,-0.80,-15.46,34.50,13.70,163.0,-0.53,0.88,42.16,OX40N,NTT,BANKS
45,KANSAINER,340.0,-67.61,54.0,H-SC,1.45,223263.0,-46404.0,82741.0,1.15,-17.21,37.06,13.47,138.0,-0.56,1.63,12.92,XY24,NTT,PAINTS
77,TTKPRESTIG,770.0,92.53,35.0,M-SC,2.74,82988.0,-17789.0,17884.0,-3.02,-17.65,21.55,0.09,245.0,-0.99,0.61,9.03,OX40N,NTT,DURABLES
18,CERA,9475.0,-22.67,34.0,H-SC,1.85,140783.0,-35120.0,77135.0,-0.11,-19.97,54.79,23.89,149.0,-0.46,1.03,21.58,OX40N,NTT,CERAMICS
66,SIS,528.0,2081.35,40.0,H-SC,4.23,88026.0,-23006.0,46610.0,-0.26,-20.72,52.95,21.26,156.0,-0.49,0.64,18.93,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,44.35,54.0,M-MC,11.15,227425.0,2463.0,165202.0,-3.26,1.09,72.64,74.53,192.0,0.01,1.66,33.70,XY24,BTT,STEEL
14,BLUESTARCO,2080.00,9.48,47.0,H-MC,3.82,188890.0,24220.0,19116.0,-1.99,14.71,10.12,26.31,89.0,1.27,1.38,23.86,X40N,NTT,AC
84,VOLTAS,1530.00,-3.46,40.0,H-MC,3.03,202440.0,10698.0,27066.0,-1.66,5.58,13.37,19.69,99.0,0.40,1.48,12.76,XY25,NTT,AC
17,CAMS,5211.76,-5.83,44.0,H-SC,1.76,107139.0,5135.0,38795.0,-1.09,5.03,36.21,43.06,122.0,0.13,0.78,23.01,X40N,ATH,MISC
79,UNIONBANK,163.00,-14.73,52.0,M-LC,6.82,154007.0,13167.0,31325.0,-2.39,9.35,20.34,31.59,66.0,0.42,1.13,38.31,XY24,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-46.15,48.0,H-MC,10.91,106853.0,6349.0,22760.0,-1.95,6.32,21.30,28.96,79.0,0.28,0.78,42.43,MH,ATH,METALS
85,WHIRLPOOL,2270.00,-45.08,31.0,M-SC,1.53,92918.0,1420.0,77336.0,-0.81,1.55,83.23,86.07,223.0,0.02,0.68,34.93,XR,NTT,DURABLES
11,BANKINDIA,190.00,-29.19,54.0,H-MC,8.21,181422.0,1614.0,110794.0,-1.50,0.90,61.07,62.52,88.0,0.01,1.33,35.29,XR,NTT,BANKS
37,INDIAMART,4850.92,-55.20,25.0,H-SC,10.23,125032.0,1696.0,132071.0,-0.61,1.38,105.63,108.45,119.0,0.01,0.91,24.04,AR,ATH,MISC
86,WIPRO,420.00,-14.70,38.0,M-LC,5.50,150040.0,-905.0,110354.0,-1.06,-0.60,73.55,72.51,53.0,-0.01,1.10,5.39,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,JIOFIN,387.00,-6.09,28.0,H-LC,12.95,255205.0,-12160.0,77608.0,-1.61,-4.55,30.41,24.48,48.0,-0.16,1.87,47.92,XY24,BTT,FINANCE
5,ASIANPAINT,3465.66,-17.77,29.0,H-LC,7.04,207715.0,-44053.0,97273.0,-1.81,-17.50,46.83,21.13,27.0,-0.45,1.52,11.73,X40,ATH,PAINTS
74,TCS,4476.75,-31.26,29.0,H-LC,12.07,273775.0,-72181.0,147045.0,-1.52,-20.86,53.71,21.64,1.0,-0.49,2.00,0.00,X40,ATH,IT
83,VBL,671.64,-20.80,30.0,H-LC,8.20,285186.0,-30656.0,143334.0,-1.07,-9.71,50.26,35.67,5.0,-0.21,2.08,2.80,X40N,ATH,FMCG
19,CIPLA,1795.00,-23.21,36.0,H-LC,4.55,205103.0,603.0,40815.0,-0.83,0.29,19.90,20.25,10.0,0.01,1.50,9.05,X40N,BTT,PHARMA


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.2,-7.23,41.0,H-LC,1.68,229770.0,-16381.0,95584.0,-2.07,-6.65,41.60,32.18,16.0,-0.17,1.68,26.99,X200,ATH,IT
42,ITC,452.0,-38.77,45.0,H-LC,2.15,197972.0,-2166.0,23064.0,1.19,-1.08,11.65,10.44,4.0,-0.09,1.45,4.42,X40,NTT,FMCG
65,SIEMENS,4671.5,-3.47,41.0,H-LC,2.63,155000.0,-31095.0,78570.0,-1.78,-16.71,50.69,25.51,15.0,-0.40,1.13,14.22,AR,ATH,ELECTRICAL
57,RELIANCE,1533.0,-14.04,45.0,H-LC,4.07,215327.0,4961.0,23815.0,0.58,2.36,11.06,13.68,37.0,0.21,1.57,19.27,XY25,NTT,REFINERIES
51,NESTLEIND,1377.0,-11.53,42.0,H-LC,4.33,272095.0,6669.0,50120.0,-0.91,2.51,18.42,21.40,11.0,0.13,1.99,10.03,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-31.26,29.0,H-LC,12.07,273775.0,-72181.0,147045.0,-1.52,-20.86,53.71,21.64,1.0,-0.49,2.00,0.00,X40,ATH,IT
80,UNITDSPR,1644.00,-14.51,43.0,H-LC,7.16,224030.0,-10114.0,58741.0,-1.15,-4.32,26.22,20.77,86.0,-0.17,1.64,2.30,X40N,NTT,BREWERIES
83,VBL,671.64,-20.80,30.0,H-LC,8.20,285186.0,-30656.0,143334.0,-1.07,-9.71,50.26,35.67,5.0,-0.21,2.08,2.80,X40N,ATH,FMCG
42,ITC,452.00,-38.77,45.0,H-LC,2.15,197972.0,-2166.0,23064.0,1.19,-1.08,11.65,10.44,4.0,-0.09,1.45,4.42,X40,NTT,FMCG
1,ABB,7934.00,-41.03,40.0,H-LC,7.98,246240.0,-15379.0,134595.0,-1.36,-5.88,54.66,45.57,7.0,-0.11,1.80,4.78,AR,NTT,ELECTRICAL


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,-3.47,41.0,H-LC,2.63,155000.0,-31095.0,78570.0,-1.78,-16.71,50.69,25.51,15.0,-0.40,1.13,14.22,AR,ATH,ELECTRICAL
42,ITC,452.00,-38.77,45.0,H-LC,2.15,197972.0,-2166.0,23064.0,1.19,-1.08,11.65,10.44,4.0,-0.09,1.45,4.42,X40,NTT,FMCG
19,CIPLA,1795.00,-23.21,36.0,H-LC,4.55,205103.0,603.0,40815.0,-0.83,0.29,19.90,20.25,10.0,0.01,1.50,9.05,X40N,BTT,PHARMA
5,ASIANPAINT,3465.66,-17.77,29.0,H-LC,7.04,207715.0,-44053.0,97273.0,-1.81,-17.50,46.83,21.13,27.0,-0.45,1.52,11.73,X40,ATH,PAINTS
57,RELIANCE,1533.00,-14.04,45.0,H-LC,4.07,215327.0,4961.0,23815.0,0.58,2.36,11.06,13.68,37.0,0.21,1.57,19.27,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-31.26,29.0,H-LC,12.07,273775.0,-72181.0,147045.0,-1.52,-20.86,53.71,21.64,1.0,-0.49,2.00,0.00,X40,ATH,IT
40,INFY,2275.00,-19.96,42.0,H-LC,8.12,311491.0,-1397.0,173096.0,-1.51,-0.45,55.57,54.87,3.0,-0.01,2.28,6.17,X40,BTT,IT
42,ITC,452.00,-38.77,45.0,H-LC,2.15,197972.0,-2166.0,23064.0,1.19,-1.08,11.65,10.44,4.0,-0.09,1.45,4.42,X40,NTT,FMCG
83,VBL,671.64,-20.80,30.0,H-LC,8.20,285186.0,-30656.0,143334.0,-1.07,-9.71,50.26,35.67,5.0,-0.21,2.08,2.80,X40N,ATH,FMCG
1,ABB,7934.00,-41.03,40.0,H-LC,7.98,246240.0,-15379.0,134595.0,-1.36,-5.88,54.66,45.57,7.0,-0.11,1.80,4.78,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-45.67,43.0,L-SC,41.35,79008.0,-34541.0,74591.0,-2.88,-30.42,94.41,35.27,268.0,-0.46,0.58,93.37,XR,NTT,HOTELS
61,SAMMAANCAP,326.00,-171.59,56.0,M-SC,5.87,82068.0,-20142.0,113533.0,0.27,-19.71,138.34,91.37,208.0,-0.18,0.60,33.22,XY25,NTT,FINANCE
3,ALKYLAMINE,4546.37,-7.53,39.0,H-SC,11.43,86043.0,-14920.0,109447.0,-1.12,-14.78,127.20,93.63,148.0,-0.14,0.63,28.16,SR,ATH,CHEMICALS
49,MASFIN,398.61,-19.60,40.0,H-SC,12.51,91500.0,-6480.0,28081.0,-1.23,-6.61,30.69,22.05,152.0,-0.23,0.67,33.00,XR,ATH,FINANCE
50,NATIONALUM,244.55,-46.15,48.0,H-MC,10.91,106853.0,6349.0,22760.0,-1.95,6.32,21.30,28.96,79.0,0.28,0.78,42.43,MH,ATH,METALS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1163.16,63.0,M-SC,12.65,171802.0,-9341.0,54805.0,-1.23,-5.16,31.90,25.10,235.0,-0.17,1.26,37.68,XY24,NTT,HEALTHCARE
61,SAMMAANCAP,326.00,-171.59,56.0,M-SC,5.87,82068.0,-20142.0,113533.0,0.27,-19.71,138.34,91.37,208.0,-0.18,0.60,33.22,XY25,NTT,FINANCE
71,TANLA,1963.11,-19.92,55.0,H-SC,16.98,215072.0,-55689.0,385646.0,-2.47,-20.57,179.31,121.86,133.0,-0.14,1.57,68.25,AR,ATH,IT
81,VAIBHAVGBL,521.00,52.62,50.0,H-SC,5.63,132478.0,-50297.0,161371.0,-2.54,-27.52,121.81,60.77,125.0,-0.31,0.97,18.66,XR,NTT,JEWELLERY
7,ATULAUTO,844.00,3570.27,48.0,M-SC,7.10,114330.0,-33038.0,84856.0,-1.47,-22.42,74.22,35.16,236.0,-0.39,0.84,16.78,XY24,NTT,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.28
1,25,43.80
2,50,74.97


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.87
LC    32.50
MC    23.64
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.39
X40      14.78
X40N     11.36
XR       11.17
XY25     10.17
AR        8.29
OX40N     7.89
X200      1.68
MH        1.61
X5K       1.44
SR        1.23
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.37
H-LC    25.97
H-MC    20.70
M-SC    13.03
M-LC     5.47
M-MC     2.64
L-SC     1.47
L-LC     1.06
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.07,-6.61,41.80
IT,12.90,-19.22,84.23
FINANCE,10.05,-19.36,71.44
MISC,6.99,-20.05,86.80
BANKS,6.16,-18.62,81.03
PAINTS,5.69,-18.48,36.15
ELECTRICAL,5.45,-10.31,49.02
INSURANCE,3.78,-6.84,46.78
AC,3.57,0.84,15.76


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3226843.0,22
XR,1385279.0,14
AR,1196326.0,9
X40,826158.0,10
X40N,649383.0,9
OX40N,566777.0,10
XY25,448271.0,7
SR,253784.0,2
X5K,120910.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3775787.0,29
M-SC,1463544.0,17
H-MC,1273734.0,15
H-LC,1266640.0,15
M-LC,403633.0,4
M-MC,316393.0,2
L-SC,261177.0,3
L-MC,61595.0,1
L-LC,39739.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1250473.0      6
M-SC       XY24       837330.0      7
H-SC       AR         812412.0      5
           XR         804676.0      7
H-MC       XY24       572670.0      4
H-LC       X40        522136.0      5
H-SC       X40N       319179.0      4
M-MC       XY24       316393.0      2
H-SC       OX40N      265096.0      4
           SR         253784.0      2
H-LC       X40N       242890.0      3
H-MC       X40        227520.0      4
H-LC       AR         213165.0      2
H-MC       XY25       179742.0      2
L-SC       XR         172505.0      2
M-LC       XY24       172369.0      2
M-SC       AR         170749.0      2
           OX40N      140075.0      4
           XR         125355.0      2
M-LC       X5K        120910.0      1
H-LC       XY25       115257.0      3
M-SC       XY25       113533.0      1
H-MC       XR         110794.0      1
M-LC       XR         110354.0      1
H-LC       X200        95584.0      1
L-SC       OX40N       88672.0      1
H-MC       X40N        87314.0      2
H-LC       XY24        77608.0      1
M-SC       X40         76502.0      1
H-MC       OX40N       72934.0      1
H-SC       MH          70167.0      1
L-MC       XR          61595.0      1
L-LC       XY25        39739.0      1
H-MC       MH          22760.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 40.0 seconds
